# Python OOPs — Foundations & Practice

This interactive notebook covers classes, objects, inheritance, polymorphism, encapsulation, and related pillars for building **modular** and **reusable** programs in Python.

> Run each code cell with **Shift+Enter**. Experiment by changing values to see how behavior changes.

## Introduction to Object-Oriented Programming

**OOP** is a programming paradigm that organizes software around **objects**—bundles of data (attributes) and behavior (methods).  
Key goals: **modularity**, **reusability**, **abstraction**, and **maintainability**.

**Core pillars**
- **Encapsulation:** bundle data + behavior; hide internal details
- **Abstraction:** expose only necessary concepts (e.g., interfaces/abstract classes)
- **Inheritance:** reuse behavior by deriving subclasses
- **Polymorphism:** same interface, different implementations

## Why OOP & Evolution of Python (high level)

Python supports **multiple paradigms**—procedural, functional, and object-oriented.  
OOP in Python embraces **duck typing** and **dynamic typing**, making it flexible and expressive.

You can mix styles: write small functional utilities and also design classes that model your domain.

## Class and Objects in Python

A **class** defines a blueprint. An **object** (instance) is a concrete realization of that blueprint.

In [ ]:
class Car:
    # Class attribute (shared by all instances unless shadowed)
    category = "vehicle"
    
    def __init__(self, brand, model, year):
        # Instance attributes
        self.brand = brand
        self.model = model
        self.year = year
    
    def info(self):  # Instance method (first parameter is usually `self`)
        return f"{self.year} {self.brand} {self.model} ({self.category})"

# Create objects
c1 = Car("Toyota", "Corolla", 2020)
c2 = Car("Tesla", "Model 3", 2023)

print(c1.info())
print(c2.info())

## Understanding `self` with a General Example

`self` refers to the **current instance**. It's how methods access instance attributes and other methods.

In [ ]:
class Counter:
    def __init__(self):
        self.value = 0
    
    def increment(self, step=1):
        # `self` points to the specific object
        self.value += step
        return self.value

a = Counter()
b = Counter()
print(a.increment(), a.increment())
print(b.increment(5))

## Adding parameters using in-built `@classmethod`

A `@classmethod` receives the class (`cls`) instead of the instance.  
Common use: **alternative constructors** that parse parameters or provide named construction.

In [ ]:
from datetime import date

class Employee:
    def __init__(self, name, start_date):
        self.name = name
        self.start_date = start_date
    
    @classmethod
    def from_year(cls, name, year, month=1, day=1):
        # Alternative constructor using class reference `cls`
        return cls(name, date(year, month, day))

e1 = Employee("Ava", date(2024, 6, 15))
e2 = Employee.from_year("Noah", 2025)
print(e1.name, e1.start_date)
print(e2.name, e2.start_date)

## Constructors (`__init__`) and Destructors (`__del__`)

- **Constructor**: `__init__(self, ...)` initializes newly created objects.
- **Destructor**: `__del__(self)` is called when the object is about to be destroyed (non-deterministically by GC).  
  > Avoid side effects in `__del__`—use context managers or explicit cleanup when possible.

In [ ]:
class Resource:
    def __init__(self, name):
        self.name = name
        print(f"[init] Acquiring resource: {self.name}")
    
    def use(self):
        print(f"Using {self.name}")
    
    def __del__(self):
        # Called at interpreter shutdown or when refcount hits zero (timing is not guaranteed)
        print(f"[del] Releasing resource: {self.name}")

res = Resource("file-handle")
res.use()

# Note: You may or may not see __del__ messages immediately depending on environment timing.

## Classes and Interfaces

Python doesn't have interfaces as a distinct keyword, but you can model interfaces using:
- **Abstract Base Classes (ABCs)** with `abc.ABC` and `@abstractmethod`
- **Protocols** (from `typing`) for structural subtyping (duck typing)

In [ ]:
from abc import ABC, abstractmethod

class Notifier(ABC):
    @abstractmethod
    def send(self, message: str) -> None:
        pass

class EmailNotifier(Notifier):
    def send(self, message: str) -> None:
        print(f"Email -> {message}")

class SMSNotifier(Notifier):
    def send(self, message: str) -> None:
        print(f"SMS -> {message}")

def alert_system(notifier: Notifier, msg: str):
    notifier.send(msg)

alert_system(EmailNotifier(), "Server down!")
alert_system(SMSNotifier(), "CPU usage high!")

## Calling Method from Another Class — Composition Example

**Composition** lets a class use another class as a component.

In [ ]:
class Engine:
    def start(self):
        return "Engine started"

class Vehicle:
    def __init__(self):
        self.engine = Engine()  # composed object
    
    def drive(self):
        status = self.engine.start()  # call method from another class
        return f"{status} — Vehicle moving"

v = Vehicle()
print(v.drive())

## Summary of Python OOPs — Part 1

- Defined classes/objects, learned `self`
- Constructors (`__init__`) and cautions around destructors (`__del__`)
- Used `@classmethod` for alternative constructors
- Modeled interfaces using ABCs
- Practiced composition (calling methods across classes)

## Variables in Python — Part 1

- **Instance variables**: specific to an object (e.g., `self.x`)
- **Class variables**: shared across instances unless shadowed
- **Name conventions**: `_protected`, `__private` (name mangling), public (no underscore)

In [ ]:
class Demo:
    shared = []   # class variable
    def __init__(self, value):
        self.value = value   # instance variable

a = Demo(10)
b = Demo(20)
a.shared.append("A")   # modifies the single shared list
print("shared via a:", a.shared)
print("shared via b:", b.shared)
print("a.value, b.value:", a.value, b.value)

## Variables in Python — Part 2 (private-ish & properties)

Double underscore triggers **name mangling**: `__x` becomes `_ClassName__x`.  
Use **properties** to **encapsulate** fields with getters/setters.

In [ ]:
class BankAccount:
    def __init__(self, owner, balance=0):
        self.owner = owner
        self.__balance = balance  # "private" by convention (mangled)
    
    @property
    def balance(self):
        return self.__balance
    
    @balance.setter
    def balance(self, amount):
        if amount < 0:
            raise ValueError("Balance cannot be negative")
        self.__balance = amount

acct = BankAccount("Luna", 100)
print(acct.owner, acct.balance)
acct.balance = 250
print("updated:", acct.balance)
# print(acct.__balance)  # AttributeError due to mangling
print("mangled access:", acct._BankAccount__balance)  # not recommended

## Changing Class Members in Python

Changing a **class variable** affects all instances **unless** an instance has a shadowing attribute of the same name.

In [ ]:
class Config:
    mode = "prod"

x = Config()
y = Config()
print("initial:", x.mode, y.mode, Config.mode)

Config.mode = "dev"
print("after class change:", x.mode, y.mode, Config.mode)

x.mode = "custom"  # shadow for x only
print("after instance shadow:", x.mode, y.mode, Config.mode)

## Polymorphism in Python

Python favors **duck typing**: if it **quacks** like a duck (has the method), it can be treated as one.

In [ ]:
class Dog:
    def speak(self): return "Woof"

class Cat:
    def speak(self): return "Meow"

def make_it_speak(animal):
    # any object with `.speak()` works
    print(animal.speak())

make_it_speak(Dog())
make_it_speak(Cat())

## Encapsulation in Python

Hide representation details and expose a clear API.  
Use underscored attributes and **properties** for validation & controlled access.

In [ ]:
class Temperature:
    def __init__(self, celsius):
        self._celsius = celsius
    
    @property
    def celsius(self):
        return self._celsius
    
    @celsius.setter
    def celsius(self, value):
        if value < -273.15:
            raise ValueError("Below absolute zero!")
        self._celsius = value
    
    @property
    def fahrenheit(self):
        return self._celsius * 9/5 + 32

t = Temperature(20)
print(t.celsius, t.fahrenheit)
t.celsius = 30
print(t.celsius, t.fahrenheit)

## Inheritance in Python

A subclass **inherits** attributes and methods from a base class and can extend/override them.

In [ ]:
class Shape:
    def area(self):
        raise NotImplementedError

class Rectangle(Shape):
    def __init__(self, w, h):
        self.w, self.h = w, h
    def area(self):
        return self.w * self.h

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

print(Rectangle(3,4).area())
print(Square(5).area())

## Data Abstraction in Python

Use **abstract base classes** to declare required behaviors without implementation details.

In [ ]:
from abc import ABC, abstractmethod

class Storage(ABC):
    @abstractmethod
    def save(self, data: dict) -> None: ...
    @abstractmethod
    def load(self) -> dict: ...

class MemoryStorage(Storage):
    def __init__(self): self._data = {}
    def save(self, data): self._data = dict(data)
    def load(self): return dict(self._data)

repo = MemoryStorage()
repo.save({"user": "sam"})
print(repo.load())

## Method Overriding in Python

A subclass provides its **own** implementation of a method defined in the base class.

In [ ]:
class Logger:
    def log(self, msg):
        print(f"[LOG] {msg}")

class FileLogger(Logger):
    def log(self, msg):
        # override with different behavior
        print(f"[FILE] {msg} (pretend we're writing to a file)")

Logger().log("hello")
FileLogger().log("hello")

## Method Overloading in Python (idiomatic approaches)

Python doesn't have traditional overloading by signature. Common patterns:
- Default/keyword arguments
- `*args` / `**kwargs`
- `functools.singledispatch` for function overloading by type

In [ ]:
from functools import singledispatch

@singledispatch
def stringify(x):
    return str(x)

@stringify.register
def _(x: int):
    return f"<int:{x}>"

@stringify.register
def _(x: list):
    return f"<list:{','.join(map(str,x))}>"

print(stringify(10))
print(stringify([1,2,3]))
print(stringify(3.14))

## Calling Method from Another Class — Integration Exercise

**Task:** Implement a `PaymentService` that uses a `Gateway` with a `.charge(amount)` method.  
Swap different gateways (e.g., `StripeGateway`, `PayPalGateway`) to demonstrate **polymorphism**.

In [ ]:
from abc import ABC, abstractmethod

class Gateway(ABC):
    @abstractmethod
    def charge(self, amount: float) -> bool: ...

class StripeGateway(Gateway):
    def charge(self, amount: float) -> bool:
        print(f"Stripe charging ${amount:.2f}")
        return True

class PayPalGateway(Gateway):
    def charge(self, amount: float) -> bool:
        print(f"PayPal charging ${amount:.2f}")
        return True

class PaymentService:
    def __init__(self, gateway: Gateway):
        self.gateway = gateway
    
    def pay(self, amount: float):
        ok = self.gateway.change(amount) if hasattr(self.gateway, "change") else self.gateway.charge(amount)
        return "Payment ok" if ok else "Payment failed"

print(PaymentService(StripeGateway()).pay(19.99))
print(PaymentService(PayPalGateway()).pay(49.00))

## Summary of Python OOPs — Part 2

- Differentiated instance/class variables and controlled access with properties
- Practiced encapsulation, inheritance, abstraction
- Overrode behavior; emulated overloading with defaults/`*args`/`singledispatch`
- Used composition and polymorphism in a small integration

---

## 🧪 Practice Blocks

1. **Model a Library System**
   - `Book(title, author, isbn)`
   - `Member(name)`
   - `Library` with `add_book`, `checkout(member,isbn)`, `return_book(isbn)`

2. **Shapes Polymorphism**
   - Base `Shape` with abstract `.area()`
   - `Circle`, `Triangle`, etc., implement `.area()`
   - Write a function that sums areas of any shape list

3. **Encapsulation Challenge**
   - `InventoryItem` with private quantity and price
   - Properties validate updates; raise if invalid

In [ ]:
# Your workspace — try implementing one of the practice ideas here
class TODO:
    pass

print("Ready to practice!")